Create a step-by-step plot of the HF radar radial velocities in the Ibiza Channel.

In [1]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import netCDF4
from matplotlib import rcParams
from osgeo import gdal

# Files and directories

In [119]:
imagedir = '/home/ctroupin/Data/Visible/'
imagefile = 'ibiza-channel5.tiff'
coastdir = "/home/ctroupin/Data/Coastlines/"
coastfile = "balearic_coast.dat"
figdir = "/home/ctroupin/Presentations/PresentationsLatex/20171015_SeaDataCloud_Athens/figures/"

In [51]:
rcParams.update({'font.size': 22})

# Load data 

Coastline 

In [71]:
valex = -999.
lonc, latc = np.loadtxt(os.path.join(coastdir, coastfile), usecols=(0, 1), unpack=True)
lonc[abs(lonc) > 180] = np.nan
latc[abs(latc) > 90] = np.nan

Read geotiff image

In [108]:
gtif = gdal.Open(os.path.join(imagedir, imagefile))
gtif.GetProjectionRef()

#Set the plot axis limits to the proper map coordinates.
arr = gtif.ReadAsArray()
trans = gtif.GetGeoTransform()
extent = (trans[0], trans[0] + gtif.RasterXSize*trans[1],
          trans[3] + gtif.RasterYSize*trans[5], trans[3])

Antennas positions

In [13]:
longalf = 1 + 13.15 /60.
latgalf = 38 + 57.11 / 60.
lonform = 1 + 23.32 / 60.
latform = 38 + 39.97 / 60.

Grid from the netCDF:

In [14]:
radarfile = "http://thredds.socib.es/thredds/dodsC/hf_radar/hf_radar_ibiza-scb_codarssproc001/L1/2016/dep0001_hf-radar-ibiza_scb-codarssproc001_L1_2016-01.nc"

In [95]:
with netCDF4.Dataset(radarfile) as nc:
    longrid = nc.variables['LON'][:]
    latgrid = nc.variables['LAT'][:]
    u = nc.variables['U'][0, :, :]
    v = nc.variables['V'][0, :, :]
llongrid, llatgrid = np.meshgrid(longrid, latgrid)
# Mask coordinates according to velocity field
llongrid = np.ma.masked_where(np.isnan(u), llongrid)

Data files

In [16]:
datadir = "/home/ctroupin/Data/HFradar/Radials/FORM/2016_12/"

In [17]:
datafilelist = sorted(glob.glob(os.path.join(datadir, "*.ruv")))
nfiles = len(datafilelist)
print(nfiles)

744


In [18]:
datafile1 = "/home/ctroupin/Data/HFradar/Radials/radialFORM.dat"
datafile2 = "/home/ctroupin/Data/HFradar/Radials/radialGALF.dat"

In [19]:
lon1, lat1, u1, v1 = np.loadtxt(datafile1, usecols=(0, 1, 2, 3), unpack=True)
lon2, lat2, u2, v2 = np.loadtxt(datafile2, usecols=(0, 1, 2, 3), unpack=True)

Define the projection

In [75]:
m = Basemap(projection='cyl',llcrnrlat=37.97,urcrnrlat=39.3,\
            llcrnrlon=0.,urcrnrlon=1.48,resolution='i')

In [135]:
formcolor = "#FFFA02"
galfcolor = "#02FFAE"
i = 0
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(111)
ax.set_xlim(-.2, 1.75)
ax.set_ylim(37.95, 39.6)
plt.xticks([])
plt.yticks([])

# Add the satellite image
plt.imshow(arr[:3,:,:].transpose((1, 2, 0)), extent=extent, zorder=2, alpha=1)
i += 1
ii = str(i).zfill(2)
plt.savefig(os.path.join(figdir, "radar{0}.png".format(ii)), dpi=300, bbox_inches="tight", pad_inches = 0)

# Add antennas positions
plt.plot(longalf, latgalf, '*', ms=10, color=galfcolor, zorder=4)
plt.plot(lonform, latform, '*', ms=10, color=formcolor, zorder=4)
i += 1
ii = str(i).zfill(2)
plt.savefig(os.path.join(figdir, "radar{0}.png".format(ii)), dpi=300, bbox_inches="tight", pad_inches = 0)

# Add vectors
q1 = plt.quiver(lon1, lat1, u1, v1, scale=1000, width=.002, headwidth=5, color=formcolor, zorder=4, alpha=.7)
i += 1
ii = str(i).zfill(2)
plt.savefig(os.path.join(figdir, "radar{0}.png".format(ii)), dpi=300, bbox_inches="tight", pad_inches = 0)

q2 = plt.quiver(lon2, lat2, u2, v2, scale=1000, width=.002, headwidth=5, color=galfcolor, zorder=4, alpha=.7)
i += 1
ii = str(i).zfill(2)
plt.savefig(os.path.join(figdir, "radar{0}.png".format(ii)), dpi=300, bbox_inches="tight", pad_inches = 0)

# Add grid positions
q1.set_alpha(.25)
q2.set_alpha(.25)
scat = plt.scatter(llongrid, llatgrid, s=5, c='w', zorder=6, cmap=plt.cm.gray_r, )
i += 1
ii = str(i).zfill(2)
plt.savefig(os.path.join(figdir, "radar{0}.png".format(ii)), dpi=300, bbox_inches="tight", pad_inches = 0)

# Final vectors
scat.set_visible(False)
plt.quiver(llongrid, llatgrid, u, v, scale=10, color="w", zorder=8)
i += 1
ii = str(i).zfill(2)
plt.savefig(os.path.join(figdir, "radar{0}.png".format(ii)), dpi=300, bbox_inches="tight", pad_inches = 0)
plt.close()